In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

import sys
sys.path.append('/home/cat/code/gerbil/utils/')

#
import numpy as np

#
from visualize import Visualize
from track import Track


Autosaving every 180 seconds


In [31]:
######################################################## 
################ GENERATE FIXED TRACK ##################
########################################################

# 
fname_slp = '/media/cat/256GB/dan/id_switch/id_switch_correction_evaluation/2020_07_30_16_05_54_197269_compressed_Day_350fixed.slp'
track = Track(fname_slp)
track.track_type = 'features'

###### parameters for computing body centroid #######
track.use_dynamic_centroid = True   # True: alg. serches for the first non-nan value in this body order [2,3,1,0,4,5]
                                     # - advantage: much more robust to lost features
                                     # False: we fix the centroid to a specific body part
                                     # - advantage less jitter for some applications
track.centroid_body_id = [2]         # if centroid flag is False; we use this body part instead

##### run track fixer #######
track.fix_all_tracks()

##### join spatially close but temporally distant chunks #####
if True:
    #
    track.memory_interpolate_tracks_spine()

##### save the fixed spines will overwrite the previous/defatul spine values####
track.save_centroid()
    
#
print ("Done...")

interpolate betweensmall bits: 100%|██████████| 28800/28800 [00:00<00:00, 217785.36it/s]


... Making tracks chunks...
... Fixing tracks...
UPDATING TRACKS


interpolate betweensmall bits: 100%|██████████| 28800/28800 [00:00<00:00, 229641.37it/s]


saving centroid
Done...


In [32]:
####################################################
########## MAKE PLOTS OF SPINE CENTRES #############
####################################################

#
def plot_track(track, animal_id):
    
    #
    plt.plot(track[:,animal_id,0],
             track[:,animal_id,1],
             #label=str(k),

            )

    plt.xlim(0,1000)
    plt.ylim(0,800)

#
animal_id = 0
    
#    
track = np.load('/media/cat/256GB/dan/id_switch/id_switch_correction_evaluation/2020_07_30_16_05_54_197269_compressed_Day_original_spine.npy')
plot_track(track, animal_id)

#    
track = np.load('/media/cat/256GB/dan/id_switch/id_switch_correction_evaluation/2020_07_30_16_05_54_197269_compressed_Day_350fixed_spine.npy')
plot_track(track, animal_id)


In [57]:

def check_switch(tracks, track, animal_id, threshold=10):
    
    #
    tracks = np.delete(tracks,animal_id, axis=0)

    #
    dists = np.linalg.norm(tracks-track, axis=1)
    if np.nanmin(dists)<threshold:
        return True
    
    return False
    
    
#
def find_bouts(track,animal_id):
    from tqdm import trange
    bouts = []
    track1 = track[:,animal_id]
    
    starts = []
    ends = []
    switches=0
    if np.isnan(track1[0,0])==False:
        starts.append(0)
    #
    for k in trange(1,track1.shape[0]-1,1):
        if np.isnan(track1[k,0]) and np.isnan(track1[k+1,0])==False:
            starts.append(k)
        #
        elif np.isnan(track1[k,0])==False and np.isnan(track1[k+1,0])==True:
            ends.append(k)
            
            # also check if the next frame is a switch
            res = check_switch(track[k+1], track1[k], animal_id)
            
            if res:
                switches+=1
            
    if len(starts)!=len(ends):
        starts=starts[:-1]
    
    bouts = np.vstack((starts,ends)).T
    #print ("bouts: ", bouts)
    
    return bouts, switches


#
animal_ids = np.arange(6)
plt.figure()

# 
for ctr, animal_id in enumerate(animal_ids):

    #
    track = np.load('/media/cat/256GB/dan/id_switch/id_switch_correction_evaluation/2020_07_30_16_05_54_197269_compressed_Day_original_spine.npy')
    bouts1,switches1 = find_bouts(track, animal_id)

    track = np.load('/media/cat/256GB/dan/id_switch/id_switch_correction_evaluation/2020_07_30_16_05_54_197269_compressed_Day_350fixed_spine.npy')
    bouts2,switches2 = find_bouts(track, animal_id)
    
    # 
    ax=plt.subplot(2,2,1)
    
    if ctr==0:
        plt.bar(ctr, bouts1.shape[0], 0.4, color='black', label='original')
        plt.bar(ctr+0.4, bouts2.shape[0], 0.4, color='blue', label='350fixed')
    else:
        plt.bar(ctr, bouts1.shape[0], 0.4, color='black')
        plt.bar(ctr+0.4, bouts2.shape[0], 0.4, color='blue')
    plt.legend()
    plt.ylabel("# of segments")
    
    #
    ax2=plt.subplot(2,2,2)
    max_ = 1800
    plt.title("350fixed - original")
    #plt.semilogy()
    diffs1 = bouts1[:,1]-bouts1[:,0]
    y1= np.histogram(diffs, bins=np.arange(0,max_,10))
    
    #
    diffs2 = bouts2[:,1]-bouts2[:,0]
    y2= np.histogram(diffs2, bins=np.arange(0,max_,10))
    
    plt.plot(y1[1][:-1],y2[0]-y1[0], label=str(ctr))
    plt.plot([0,max_],[0,0],'--',c='grey')
    plt.xlabel("duration of bout")
    plt.ylabel("# of bouts")
    plt.legend()
    
    #
    ax3=plt.subplot(2,2,3)
    plt.bar(ctr, diffs1.sum()/track.shape[0], 0.4, color='black')
    plt.bar(ctr+0.4, diffs2.sum()/track.shape[0], 0.4, color='blue')  
    plt.ylabel("total duration of tracked bouts ")
    plt.xlabel("animal id")
    
    ax3=plt.subplot(2,2,4)
    plt.bar(ctr, switches1, 0.4, color='black')
    plt.bar(ctr+0.4, switches2, 0.4, color='blue')  
    plt.ylabel("Likely ID switches")
    plt.xlabel("animal id")
#
plt.show()



100%|██████████| 28800/28800 [00:00<00:00, 64193.41it/s]


[nan nan]
[[         nan          nan]
 [732.66070557 144.16307068]
 [271.63406372 364.27807617]
 [         nan          nan]
 [         nan          nan]]
[nan nan]
[[         nan          nan]
 [732.28009033 140.12745667]
 [271.33508301 364.21456909]
 [         nan          nan]
 [         nan          nan]]
[nan nan]
[[         nan          nan]
 [720.37005615 144.15052795]
 [268.88034058 364.16134644]
 [         nan          nan]
 [520.30529785 551.73413086]]
[519.49688721 151.09466553]
[[         nan          nan]
 [716.69995117 143.34922791]
 [268.7923584  364.45471191]
 [         nan          nan]
 [520.17321777 551.73583984]]
[519.50750732 151.03465271]
[[         nan          nan]
 [719.87268066 136.60595703]
 [264.06903076 367.54333496]
 [         nan          nan]
 [520.29669189 551.7802124 ]]
[519.83166504 151.00440979]
[[         nan          nan]
 [716.19372559 139.77746582]
 [259.80831909 368.03771973]
 [         nan          nan]
 [520.32543945 551.68341064]]
[519.81970


KeyboardInterrupt



In [13]:
####################################################
########## MAKE MOVIES OF SPINE CENTRES ############
####################################################
print ("")
print ("Making movie...")

#
vis = Visualize()
fps = 25
start = 0*fps       # frame to start in seconds
end = 60*fps        # frame to end in seconds
fps = 25

#
fname_video = '/media/cat/256GB/dan/testing_track_cleanup_code/2020_07_26_18_35_09_660833_compressed_Day_huddle/2020_07_26_18_35_09_660833_compressed.mp4'
fname_video_extension = '_fixed'  # the extension to be added to the filename

# Note; thre is also a version of this that plots all body parts
# - this function takes in only a single animal location 
shrink = 1 # shrink image factor

fname_track = '/media/cat/256GB/dan/testing_track_cleanup_code/2020_07_26_18_35_09_660833_compressed_Day_huddle/2020_07_26_18_35_09_660833_compressed_Day_huddle_multi_track_huddles.npy'
track = np.load(fname_track)
print ("track shape: ", track.shape)
#print (track.tracks_spine.shape)
#
vis.make_video_centroid(track,                # visualize centres locations
                        fname_video,
                        fname_video_extension,
                        start,
                        end,
                        fps,
                        shrink) 

# 
print ("DONE...")

  0%|          | 5/1500 [00:00<00:32, 46.07it/s]


Making movie...
track shape:  (28802, 10, 2)
width, heigh:  1280.0 1024.0
Histories:  (10, 5, 2)


100%|██████████| 1500/1500 [00:28<00:00, 52.92it/s]

DONE...


In [6]:
fname_slp = '/media/cat/256GB/dan/simba/old/2020_07_31_20_15_17_668937_compressed_defished_shrink_cropped.slp'
track = Track(fname_slp)
track.track_type = 'features'
###### parameters for computing body centroid #######
track.use_dynamic_centroid = True   # True: alg. serches for the first non-nan value in this body order [2,3,1,0,4,5]
                                     # - advantage: much more robust to lost features
                                     # False: we fix the centroid to a specific body part
                                     # - advantage less jitter for some applications
track.centroid_body_id = [2]    

#
track.load_tracks()

print (track.tracks.shape)

(28802, 2, 6, 2)


In [7]:
animal_id = 0

t = track.tracks[:,animal_id]
print(t.shape)

plt.figure()

plt.scatter(t[:,1,0],
         t[:,1,1])



plt.show()

(28802, 6, 2)
